In [ ]:
import json
import requests
import string
from collections import Counter


ACCESSTOKEN  = 'CAACEdEose0cBAFeKLIpY8sVvZBwvPwVyYfPHmqHA5khSaVnm7cuF7ghzmxW5tSoh51gi3AEtYzUpICqUZBAu5ZB1ayLVpYdMCcBgIy9suAi2WWpd3LyD8ZAUvmv6t0UbOqZCtofmLqJs6kznbPa7O1kggoaVIHOHlBhZA3hZCMBQFDdH5AvzguWEjcaAGVImJCiIZAwppZCmZBtn55onTHoYIW'
PostUrl ='https://graph.facebook.com/v2.5/10154049289398949_10154049291803949/likes?limit=1000&access_token='+ACCESSTOKEN

res = requests.get(PostUrl)
print res.text
jd = json.loads(res.text)

PostID= jd['data'][0]['id']
#print jd['data'][0]['like_count']
len(jd['data'])

In [ ]:
import time
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.72.42;DATABASE=FB_IMDB;UID=sa;PWD=scucc')
cursor = conn.cursor()
cursor.execute("select * from [FBComment] where PostNO>0 and LikeCount>0  order by PostNO,CommentNO asc")

rows = cursor.fetchall()


ACCESSTOKEN  = 'CAACEdEose0cBANgr2kyqSK4WX1aBtkmNaNRlh8FitjFKLXaXZAWSdJDV3Yxb2rFkXt7sKtUR6zQsMH46w0ZBKBrC9icnwIYcEMlyNQdLFvlju3pdaQlOvTFQJUloXTCiaw3vA9tP7eoXnZC578Dn8r6ZBHmWNwWWmY4VnLIHzYipFzmFRCEYlemsJK1Ps7F5GnytzKvRAecQE3UOSWtX'
for row in rows :
    LikeNO=1
    PostUrl ='https://graph.facebook.com/v2.5/{}/likes?pretty=0&limit=1000&access_token='+ACCESSTOKEN
    CommentID= row.CommentID
    CommentNO= row.CommentNO
    PostNO=row.PostNO
    res = requests.get(PostUrl.format(CommentID))
    jd = json.loads(res.text)
    
   
    
    if 'Unsupported get request'in res.text :
        insertSQL2()
        continue
    if 'data' in res.text and  'paging'not in res.text:
        insertSQL2()
        continue
    while 'paging'not in res.text  :
        i=1
        time.sleep(1)
        print PostUrl.format(CommentID)
        res = requests.get(PostUrl.format(CommentID))
        jd = json.loads(res.text)
    while 'next' in jd['paging']:
        for data in jd['data']:
            UserName= data['name']
            UserID= data['id']
           
            insertSQL()
            if(LikeNO%1000==0):
                url=jd['paging']['next']
                #time.sleep(0.1)
                res=requests.get(url)
                jd = json.loads(res.text)
            LikeNO+=1
            while 'paging' not in res.text:
                i=1
                time.sleep(1)
                print PostUrl.format(CommentID)
                res = requests.get(PostUrl.format(CommentID))
                jd = json.loads(res.text)  
        
    for data in jd['data']:
        UserName= data['name']
        UserID= data['id']
        insertSQL()
        LikeNO+=1
    

In [11]:
def insertSQL():
    command="INSERT INTO [FBCommentLike] (PostNO,CommentNO,LikeNO,UserID,UserName) VALUES (?,?,?,?,?)"

    cursor.execute(command,(PostNO,CommentNO,LikeNO,UserID,UserName))
    conn.commit()

# rows = cursor.fetchall()
# for row in rows:
#     print row.id

In [7]:
def insertSQL2():
    command="INSERT INTO FBCommentLike (PostNO) VALUES (?)"

    cursor.execute(command,(PostNO))
    conn.commit()